# Segmenting and Clustering Neighborhoods in Toronto

## Import various libraries that will be used 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins
from folium.plugins import MarkerCluster

print('Libraries imported.')

Libraries imported.


In [2]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


#### Webscrape website for postal Codes in Canada and print shape of table

In [3]:
#read table using pandas, 
url = 'http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url)
df = table[0]


In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


##### Replaced not assigned with NAN and checked to make sure that Neighbourhood has a corresponding Borough

In [8]:
#replace 'Not assigned' with 'NaN'
df1 = df[df.Borough != "Not assigned"]
df2 = df1.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df2.reset_index(inplace=True)        

##### Replace neighbourhood now assign with a Borough name

##### Dropped NaN from table

In [15]:
df2['Neighbourhood']= np.where(df2['Neighbourhood']== 'Not assigned', df2['Borough'], df2['Neighbourhood'] )
df2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [19]:
df2.shape

(103, 3)

In [22]:
lat_long = pd.read_csv('/Dropbox/Data Science/TABLES/Geo.csv')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Merge the two tables

In [25]:
lat_long.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df_main = pd.merge(df2, lat_long, on='Postcode')

In [26]:
df_main

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


##### Rows where Toronto is in the name of the Borough

In [27]:
df3 = df_main[df_main['Borough'].str.contains('Toronto')]

In [28]:
df3

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259


In [29]:
df3.shape

(39, 5)

##### Cluster of neighbour hoods with Toronto in the name

In [44]:
mapof_toronto = folium.Map(location=[43.654260,-79.360636], zoom_start=11)

for lat, long, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long], radius =10, popup=label, color='black', fill=True, 
    parse_html=False).add_to(mapof_toronto)

mapof_toronto                        
                        
                        